In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 22 18:47:04 2019

@author: Admin
"""


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
#import timeit
import time
import pandas as pd
from pandas import ExcelWriter
from bs4 import BeautifulSoup as bs
from collections import namedtuple
from requests_futures.sessions import FuturesSession
from urllib.parse import urljoin

driver = webdriver.Chrome(r'C:\Users\Admin\Desktop\chromedriver.exe')
# listkey để chứa từ khóa tìm kiếm
listkey=['abc','bcd','def']
# tuy thuoc vao tung trang web, từng yêu cầu, mà số lượng thành phần của nametuple tăng hoặc giảm cho hợp lý
CourseDetailed = namedtuple('CourseDetailed', 'title, company, description')
start= time.time()

In [3]:
# Hàm details để lấy thông tin từ từng trang web
def detail(response):
    # lần lượt tìm kiếm và lấy thông tin từ từng div class tương ứng
    soup = bs(response.text, 'lxml')
    tit = soup.find('div', attrs={'class':'content bt3-col-xs-12 bt3-col-sm-12 bt3-col-md-12 bt3-col-lg-12'})
    if tit== None:
        tit = soup.find('h1', attrs={'class':'H2_1pmnvep-o_O-weightNormal_s9jwp5-o_O-fontHeadline_1uu0gyz max-text-width-xl m-b-1s'})
        if tit==None :
            tit = soup.find('h2', attrs={'class':'display-5-text degree-name'})
            if tit== None:
                title='Degrees & Certificates'
            else:
                title= tit.text
        else:
            title = tit.text
    else:
        title = tit.text
    company = 'Empty()'
    des = soup.find('div', attrs={'class':'AboutCourse'})
    if des== None:
        des=soup.find('div', attrs={'class':'rc-Column bt3-col-xs-12 bt3-col-md-7 rc-MainColumn'})
        if des==None:
            des=soup.find('div',attrs={'class':'content-inner'})
            if des==None:
                des='Degrees & Certificates'
            else:
                des=des.text
        else:
            des=des.find('div')
            des=des.text
    else :
        des=des.text
    
    res=CourseDetailed(title, company, des)
    return res

In [ ]:
urls = []
for i in listkey:
    # TÌM KIẾM KEYWORD TRONG LISTKEY
    driver.get('(<TRANG WEB CẦN TÌM>')
    # TÌM DIV CLASS CỦA Ô INPUT ĐỂ THAY ĐỔI
    keyword = driver.find_element_by_xpath("//input[@id='q']")
    keyword.clear()
    keyword.send_keys(i)
    keyword.send_keys(Keys.RETURN)
    #NẾU TRANG WEB K CHO TỰ NHẬP KEY THÌ TA LẤY URLS TÌM KIẾM NHẬP VÀO LISTKEY VÀ GET(i) 
    driver.get(i)
    # KIỂM TRA NẾU TỪ KHÓA KHÔNG CÓ KẾT QUẢ NÀO THÌ BỎ QUA
    xt= driver.find_elements_by_xpath("//div[@id='no_results']")
    if len(xt)>0:
        continue
    # BẮT ĐẦU LOAD TRANG
    source_code = driver.page_source    
    soup = bs(source_code, 'lxml')
    while True:
        SCROLL_PAUSE_TIME = 0.5
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
# Wait to load page-----------------------------------------------------------------------------------------------------------------------------------------------------------
'''
HIỆN TẠI CÓ 3 DẠNG CUỘN TRANG WEB CHÍNH LÀ
----------------(1)-----------------------
Cuộn để lấy thêm bài:
- Data Science & Big Data Vietnam (fb)
- Vietnam Data Scientists (fb)
- Forum Machine Learning cơ bản (fb)
----------------(2)-----------------------
Nhấn xem thêm show thêm bài:
- https://itviec.com
- Internship.edu.vn
----------------(3)-----------------------
Sô thứ tự 1, 2, 3:
- https://www.vietnamworks.com
- https://www.glassdoor.com
- https://www.careerbuilder.com
- https://www.jobstreet.vn (edited) 
'''
        for scr in range(1000):
                driver.find_element_by_tag_name('body').send_keys(Keys.END)
                time.sleep(1.5)
                # TÌM KIẾM TẤT CẢ CÁC NÚT LOADMORE (DÀNH CHO ĐẠNG TRANG WEB Ở (2))
                a=driver.find_elements_by_class_name('ais-InfiniteHits-loadMore')
                if len(a)==0:
                    #print("      no more job with this keyword, go to the next")
                    break
                for cl in a:
                    cl.click()
# Wait to load page-----------------------------------------------------------------------------------------------------------------------------------------------------------
        #DẠNG CUỘN XUỐNG BOTTOM WEB SẼ TỰ LOAD THÊM DỮ LIỆU NHƯ (1) THÌ DÙNG :
    '''
     SCROLL_PAUSE_TIME = 0.5

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)
        
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
    
    '''
# Wait to load page-----------------------------------------------------------------------------------------------------------------------------------------------------------
        # DẠNG CLICK SỐ THỨ TỰ ĐỂ LOAD QUA TRANG KHÁC THÌ HƠI KHÁC 1 CHÚT
    #VIỆC APPEND URL VÀ LOAD TRANG PHẢI THỰC HIỆN ĐỒNG THỜI VÀ XEN KẼ.
    xt=driver.find_elements_by_link_text("Kế tiếp")
    if len(xt) == 0:
        break
    driver.find_element_by_link_text("Kế tiếp").click()
# Wait to load page-----------------------------------------------------------------------------------------------------------------------------------------------------------
#        PHẦN NÀY DÙNG ĐỂ TÌM KIẾM XÁC ĐỊNH CÓ POP-UP HAY KHÔNG -TÙY TRANG WEB
        close=driver.find_elements_by_xpath("//div[@class='lean-overlay']")
        if(len(close))>0:
            print("      !!![WARNING]:---have a fade in popup, wait a few second to close it before get the next page")
            time.sleep(1)
            driver.find_element_by_xpath("//div[@id='job_alert_modal']//a").click()
            time.sleep(1)
#            
# all work will be here------------------------------------------------------------------------------------------
        source_code = driver.page_source    
        soup = bs(source_code, 'lxml')
#        pre=soup.find('ul', attrs={'id':'jobresults'})
        jobs=soup.find_all('li', attrs={'class':'ais-InfiniteHits-item'} )
        print(len(jobs))
        #       find element have link
        for item in jobs:
            base='https://coursera.org'
            urlx=urljoin(base,item.select('a')[0]['href'] )
            urls.append(urlx)
#            print(urlx)
        break


In [11]:
# Dùng future session và hàm Details để thu thập thông tin

session = FuturesSession(max_workers=20)
futures = [url for url in urls]
    
details = [detail(url) for url in urls]
data_dict = {'url': urls, 'title': [detail.title for detail in details], 'company': [detail.company for detail in details], 'description':[detail.description for detail in details] }
data_frame = pd.DataFrame(data_dict)
end = time.time()
data_frame.to_csv('COURSERA-ORG.csv')
print(end - start)

4770.39497923851


In [12]:
nhìn lại khoảng thời gian cách đây 3 tuần: Huân đã học được khá nhiều thứ so với lúc không biết chút gì, phải bắt đầu, phải thử
    thì mới biết bản thân có thể hay không thể, nhưng nhìn vào task remaining (+ issues) và imagine về coming task thì thấy 
    bản thân em còn quá nhiều hạn chế, 2 tiếng mội ngày quả thật là không đủ cho việc tự học tự tìm hiểu và tự giải quyết vde
    em thấy bản thân mình cần cố gắng nhiều thêm nữa. 
    đa phần dữ liệu cào về, các con số nhìn thấy được chỉ là số bài/keyword/web và thời gian từng trang.
    nếu dùng để làm data visualize thì chắc chắn sẽ rất nghèo nàn và không biểu thị được nhiều thông tin.
    và em cũng rất háo hức chờ tới phần xử lý dữ liệu, để từ dữ liệu thô trở nên có ích là như thế nào <3 em muốn viết thêm, nhưng bạn ngân k cho
    

SyntaxError: invalid syntax (<ipython-input-12-9aaf1dd19b38>, line 1)